In [478]:
import torch
import random

In [460]:
cube = torch.randint(1,7,(6,3,3))
cube

tensor([[[2, 3, 6],
         [1, 5, 5],
         [2, 3, 6]],

        [[1, 5, 4],
         [4, 3, 2],
         [5, 2, 2]],

        [[4, 5, 4],
         [6, 6, 2],
         [6, 5, 1]],

        [[4, 5, 1],
         [1, 4, 4],
         [1, 2, 6]],

        [[3, 2, 6],
         [3, 4, 5],
         [3, 6, 1]],

        [[2, 4, 4],
         [6, 4, 4],
         [1, 5, 3]]])

In [461]:
faces ={0:'F',1:'R',2:'B',3:'L',4:'U',5:'D'}
faces

{0: 'F', 1: 'R', 2: 'B', 3: 'L', 4: 'U', 5: 'D'}

In [462]:
faces = {j:i for i,j in faces.items()}
faces

{'F': 0, 'R': 1, 'B': 2, 'L': 3, 'U': 4, 'D': 5}

In [463]:
def reindex_faces(cube,face):

    if face in [0,1,2,3]:
        n = 4-face
        r=torch.zeros(4,3,3)

        while n!=0 :
            n -= 1
            r[0:4] = cube[0:4].flip(0)
            r[1:4]= r[0:4][1:4].flip(0)
            cube[0:4] = r[0:4]
            cube[4] = cube[4].transpose(0,1).flip(0,)
            cube[5] = cube[5].transpose(0,1).flip(0,)
            
    else:
        s=torch.zeros(4,3,3)

        s[0] = cube[0]
        s[1] = cube[5]
        s[2] = cube[2]
        s[3] = cube[4]

        if(face == 4):
            n=1
        else:
            n=3

        while n!=0 :
            n -= 1
            s[0:4] = s[0:4].flip(0)
            s[1:4] = s[0:4][1:4].flip(0)
            cube[1] = cube[1].transpose(0,1).flip(0,)
            cube[3] = cube[3].transpose(0,1).flip(1,)

        #As on rotating the faces upside down the face opposite to front face will have modified dimensions due to rotation
        #Therefore modifying the dimension of cube[2] and cube[4] in the end to make it correct
       
        cube[0]=s[0]
        cube[5]=s[1]
        cube[2]=s[2].flip(0).flip(1)
        cube[4] = s[3].flip(0).flip(1)
                
    return cube

In [ ]:
def reindex_faces(cube,face):

    if face in [0,1,2,3]:
        n = 4-face
        r=torch.zeros(4,3,3)

        while n!=0 :
            n -= 1
            r[0:4] = cube[0:4].flip(0)
            r[1:4]= r[0:4][1:4].flip(0)
            cube[0:4] = r[0:4]
            cube[4] = cube[4].transpose(0,1).flip(0,)
            cube[5] = cube[5].transpose(0,1).flip(0,)
    else: 
        s=torch.zeros(4,3,3)

        s[0] = cube[0]
        s[1] = cube[5]
        s[2] = cube[2]
        s[3] = cube[4]

        if face == 4:
            n=1
        else:
            n=3

        while n!=0 :
            n -= 1
            s[0:4] = s[0:4].flip(0)
            s[1:4] = s[0:4][1:4].flip(0)
            cube[1] = cube[1].transpose(0,1).flip(0,)
            cube[3] = cube[3].transpose(0,1).flip(1,)

        #As on rotating the faces upside down the face opposite to front face will have modified dimensions due to rotation
        #Therefore modifying the dimension of cube[2] and cube[4] in the end to make it correct
       
        cube[0]=s[0]
        cube[5]=s[1]
        cube[2]=s[2].flip(0).flip(1)
        cube[4]=s[3].flip(0,1)

    return cube

In [464]:
def rot_sideways(cube,face,row,direction):
    

    face = faces[face]
    #rotate the cube so as to make face given above as front face
    if face!=0 :
        cube = reindex_faces(cube,face)
        
    if row == 1:
        #rotation of upper face
        if direction == 'C':
            
            #rotation of row along dimensions
            c=cube[:4,:,:]
            c[:,0,:] = c[:,0,:].flip(0)
            c[:,0,:][1:4] = c[:,0,:][1:4].flip(0)
            
            cube[0:4] = c
            
            #rotation clockwise
            cube[4] = cube[4].transpose(0,1).flip(0,)
            
        else:
            c=cube[:4,:,:]
            c[:,0,:] = c[:,0,:].flip(0)
            c[:,0,:][0:3] = c[:,0,:][0:3].flip(0)
            
            cube[0:4] = c
            
            #rotation anticlockwise
            cube[4] = cube[4].flip(0,).transpose(0,1)
            
    elif row == 3 :
        #rotation of upper face
        if direction == 'C':
            
            c=cube[:4,:,:]
            c[:,2,:] = c[:,2,:].flip(0)
            c[:,2,:][1:4] = c[:,2,:][1:4].flip(0)
        
            cube[0:4] = c
            
            #rotation clockwise
            cube[5] = cube[5].transpose(0,1).flip(1,)
        else:
            c=cube[:4,:,:]
            c[:,2,:] = c[:,2,:].flip(0)
            c[:,2,:][0:3] = c[:,2,:][0:3].flip(0)
            
            cube[0:4] = c
            
            #rotation anticlockwise
            cube[5] = cube[5].flip(1,).transpose(0,1)
            
    else:
        if direction == 'C':
            c=cube[:4,:,:]
            c[:,1,:] = c[:,1,:].flip(0)
            c[:,1,:][1:4] = c[:,1,:][1:4].flip(0)
            
            cube[0:4] = c
        else:
            c=cube[:4,:,:]
            c[:,1,:] = c[:,1,:].flip(0)
            c[:,1,:][0:3] = c[:,1,:][0:3].flip(0)
            
            cube[0:4] = c
              
    return cube,face


In [451]:
def re_reindex_faces(cube,face):
#re rotating the face to original position
    if face != 0 :
        if face == 4:
            face = face+1
        elif face == 5:
            face = face-1
        elif face == 3: 
            face = face-2
        else:
            face = face+2 
            
        cube = reindex_faces(cube,face)
        
        cube.flip(2,)
        
        if face == 5:
            cube[4] = cube[4].flip(0).flip(1)
            cube[5] = cube[5].flip(0).flip(1)
    return cube

In [465]:
cube

tensor([[[2, 3, 6],
         [1, 5, 5],
         [2, 3, 6]],

        [[1, 5, 4],
         [4, 3, 2],
         [5, 2, 2]],

        [[4, 5, 4],
         [6, 6, 2],
         [6, 5, 1]],

        [[4, 5, 1],
         [1, 4, 4],
         [1, 2, 6]],

        [[3, 2, 6],
         [3, 4, 5],
         [3, 6, 1]],

        [[2, 4, 4],
         [6, 4, 4],
         [1, 5, 3]]])

In [467]:
cube,face = rot_sideways(cube,'F',1,'C')
cube = re_reindex_faces(cube,face)
cube

tensor([[[4, 5, 4],
         [1, 5, 5],
         [2, 3, 6]],

        [[4, 5, 1],
         [4, 3, 2],
         [5, 2, 2]],

        [[2, 3, 6],
         [6, 6, 2],
         [6, 5, 1]],

        [[1, 5, 4],
         [1, 4, 4],
         [1, 2, 6]],

        [[1, 6, 3],
         [5, 4, 3],
         [6, 2, 3]],

        [[2, 4, 4],
         [6, 4, 4],
         [1, 5, 3]]])

In [285]:
def rot_upside(cube,face,row,direction):
    
    initial_face = faces[face]
    face = faces[face]
    #rotate the cube so as to make face given above as front face
    if face!=0 :
        cube = reindex_faces(cube,face)
    
    #Now turning the cube so as to apply rotate sideways function later on 
    cube[0]=cube[0].transpose(0,1).flip(1)
    cube[2]=cube[2].transpose(0,1).flip(0)
    
    s=torch.zeros(4,3,3)
    s[0] = cube[4]
    s[1] = cube[1]
    s[2] = cube[5]
    s[3] = cube[3]
    
    s[0:4] = s[0:4].flip(0)
    s[1:4]= s[0:4][1:4].flip(0)
    
    s = s.transpose(1,2).flip(2)
    
    cube[1] = s[1]
    cube[3] = s[3]
    cube[4] = s[0]
    cube[5] = s[2]
    
    #calling rot_sideways function to rotate the sides.
    # Now for up it will be Clockwise in new orientation and down it will be Antclockwise
    #face = 0 as we have already reindexed the face in the beginnin and now the face we are working on is in front
    
    if direction == 'U':
        cube,face = rot_sideways(cube,'F',row,'C')
    else:
        cube,face = rot_sideways(cube,'F',row,'AC')
    
    #After that re rotate to original upside down position 
    
    cube[0] = cube[0].flip(1).transpose(0,1)
    cube[2] = cube[2].transpose(0,1).flip(1)
    
    s=torch.zeros(4,3,3)
    
    s[0] = cube[4]
    s[1] = cube[1]
    s[2] = cube[5]
    s[3] = cube[3]
    
    s[0:4] = s[0:4].flip(0)
    s[0:3]= s[0:4][0:3].flip(0)
    
    s = s.flip(2).transpose(1,2)
    
    cube[1] = s[1]
    cube[3] = s[3]
    cube[4] = s[0]
    cube[5] = s[2]
    
    #cube = re_reindex_faces(cube,initial_face)
    #cube = cube.flip(1).flip(2)
    #cube[2] = 
    #cube[5] = cube[5].flip(1).flip(2)
    
    return cube

In [468]:
def rot_upside(cube,face,row,direction):
    
    initial_face = faces[face]
    face = faces[face]
    #rotate the cube so as to make face given above as front face
    if face!=0 :
        cube = reindex_faces(cube,face)
        
    #Now turning the cube so as to apply rotate sideways function later on 
    cube[0]=cube[0].transpose(0,1).flip(1)
    cube[2]=cube[2].transpose(0,1).flip(0)
    
    s=torch.zeros(4,3,3)
    s[0] = cube[4]
    s[1] = cube[1]
    s[2] = cube[5]
    s[3] = cube[3]
    
    s[0:4] = s[0:4].flip(0)
    s[1:4]= s[0:4][1:4].flip(0)
    
    s = s.transpose(1,2).flip(2)
    
    cube[1] = s[1]
    cube[3] = s[3]
    cube[4] = s[0]
    cube[5] = s[2]
    
    #calling rot_sideways function to rotate the sides.
    # Now for up it will be Clockwise in new orientation and down it will be Antclockwise
    #face = 0 as we have already reindexed the face in the beginnin and now the face we are working on is in front
    
    if direction == 'U':
        cube,face = rot_sideways(cube,'F',row,'C')
    else:
        cube,face = rot_sideways(cube,'F',row,'AC')
    
    #After that re rotate to original upside down position 
    
    cube[0] = cube[0].flip(1).transpose(0,1)
    cube[2] = cube[2].transpose(0,1).flip(1)
    
    s=torch.zeros(4,3,3)
    
    s[0] = cube[4]
    s[1] = cube[1]
    s[2] = cube[5]
    s[3] = cube[3]
    
    s[0:4] = s[0:4].flip(0)
    s[0:3]= s[0:4][0:3].flip(0)
    
    s = s.flip(2).transpose(1,2)
    
    cube[1] = s[1]
    cube[3] = s[3]
    cube[4] = s[0]
    cube[5] = s[2]
    
    cube = re_reindex_faces(cube,initial_face)
    
    return cube

In [469]:
cube

tensor([[[4, 5, 4],
         [1, 5, 5],
         [2, 3, 6]],

        [[4, 5, 1],
         [4, 3, 2],
         [5, 2, 2]],

        [[2, 3, 6],
         [6, 6, 2],
         [6, 5, 1]],

        [[1, 5, 4],
         [1, 4, 4],
         [1, 2, 6]],

        [[1, 6, 3],
         [5, 4, 3],
         [6, 2, 3]],

        [[2, 4, 4],
         [6, 4, 4],
         [1, 5, 3]]])

In [470]:
cube = rot_upside(cube,'F',1,'U')
cube

tensor([[[2, 5, 4],
         [6, 5, 5],
         [1, 3, 6]],

        [[4, 5, 1],
         [4, 3, 2],
         [5, 2, 2]],

        [[2, 3, 6],
         [6, 6, 5],
         [6, 5, 1]],

        [[4, 4, 6],
         [5, 4, 2],
         [1, 1, 1]],

        [[4, 6, 3],
         [1, 4, 3],
         [2, 2, 3]],

        [[1, 4, 4],
         [2, 4, 4],
         [6, 5, 3]]])

In [189]:
rot_upside(cube,'U',2,'D')

(tensor([[[2, 2, 1],
          [6, 2, 1],
          [1, 1, 3]],
 
         [[2, 2, 4],
          [6, 5, 4],
          [1, 1, 4]],
 
         [[1, 4, 2],
          [1, 1, 2],
          [4, 1, 5]],
 
         [[3, 3, 1],
          [2, 3, 3],
          [5, 2, 1]],
 
         [[3, 2, 6],
          [2, 1, 1],
          [3, 5, 4]],
 
         [[4, 4, 1],
          [5, 2, 1],
          [2, 3, 1]]]),
 0)

In [ ]:
cube = rot_upside(cube,'F',1,'U')
cube
f = ['F','R','B','L','U','D']
r = [1,2,3]
d1 = ['C', 'AC']
d2 = ['U' 'D']
cube,face = rot_sideways(cube,'F',1,'C')
cube = re_reindex_faces(cube,face)

direction = 'C' 'AC'

In [474]:
cube

tensor([[[2, 5, 4],
         [6, 5, 5],
         [1, 3, 6]],

        [[4, 5, 1],
         [4, 3, 2],
         [5, 2, 2]],

        [[2, 3, 6],
         [6, 6, 5],
         [6, 5, 1]],

        [[4, 4, 6],
         [5, 4, 2],
         [1, 1, 1]],

        [[4, 6, 3],
         [1, 4, 3],
         [2, 2, 3]],

        [[1, 4, 4],
         [2, 4, 4],
         [6, 5, 3]]])

In [476]:
def operations(cube, face, row, direction):
    if direction in ['C','AC']:
        cube,face = rot_sideways(cube,face,row,direction)
        cube = re_reindex_faces(cube,face)
    else:
        cube = rot_upside(cube,face,row,direction)
    return cube

In [477]:
possible_face_values = ['F','R','B','L','U','D']
possible_row_values = [1,2,3]
possible_directions = ['C','AC','U','D']

In [481]:
%%time
for i in range(10000):
    cube = operations(cube,random.choice(possible_face_values),random.choice(possible_row_values),random.choice(possible_directions))

Wall time: 3.21 s
